In [1]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
import os

2023-05-29 18:48:55.927016: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 18:48:57.224232: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-29 18:48:57.224283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-29 18:48:57.224288: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']
    
    
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

# create data generator
datagen = ImageDataGenerator(zoom_range=[0.9,1.1],
                             rotation_range=30,
                             brightness_range=[0.9,1.1],
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             vertical_flip=True,
                             horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [3]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nstatic const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'static const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) :

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

In [13]:
model_10 = Sequential()
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.001, momentum=0.9)

model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_Dual_3Conv.h5', save_freq='epoch')

model_10.summary()

# Train and evaluate the quantization aware model
model_10.fit(x_train,y_train, batch_size=64,epochs=400,callbacks=[checkpoint_callback],
             validation_data=(x_test, y_test))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_52 (Conv2D)          (None, 32, 32, 16)        448       
                                                                 
 conv2d_53 (Conv2D)          (None, 32, 32, 16)        2320      
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 16, 16, 16)       0         
 g2D)                                                            
                                                                 
 dropout_39 (Dropout)        (None, 16, 16, 16)        0         
                                                                 
 conv2d_54 (Conv2D)          (None, 16, 16, 32)        4640      
                                                                 
 conv2d_55 (Conv2D)          (None, 16, 16, 32)        9248      
                                                      

782/782 [==============================] - 26s 34ms/step - loss: 0.8300 - accuracy: 0.7151 - val_loss: 0.7295 - val_accuracy: 0.7493
Epoch 39/400
782/782 [==============================] - 26s 34ms/step - loss: 0.8220 - accuracy: 0.7163 - val_loss: 0.7367 - val_accuracy: 0.7440
Epoch 40/400
782/782 [==============================] - 26s 34ms/step - loss: 0.8180 - accuracy: 0.7176 - val_loss: 0.7119 - val_accuracy: 0.7556
Epoch 41/400
782/782 [==============================] - 26s 34ms/step - loss: 0.8250 - accuracy: 0.7142 - val_loss: 0.7451 - val_accuracy: 0.7365
Epoch 42/400
782/782 [==============================] - 26s 34ms/step - loss: 0.8140 - accuracy: 0.7212 - val_loss: 0.7520 - val_accuracy: 0.7415
Epoch 43/400
782/782 [==============================] - 26s 34ms/step - loss: 0.8118 - accuracy: 0.7215 - val_loss: 0.7698 - val_accuracy: 0.7340
Epoch 44/400
782/782 [==============================] - 26s 34ms/step - loss: 0.8077 - accuracy: 0.7198 - val_loss: 0.7343 - val_accuracy

782/782 [==============================] - 27s 34ms/step - loss: 0.7699 - accuracy: 0.7396 - val_loss: 0.6834 - val_accuracy: 0.7689
Epoch 95/400
782/782 [==============================] - 27s 35ms/step - loss: 0.7592 - accuracy: 0.7402 - val_loss: 0.7098 - val_accuracy: 0.7661
Epoch 96/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7454 - accuracy: 0.7464 - val_loss: 0.6586 - val_accuracy: 0.7804
Epoch 97/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7594 - accuracy: 0.7398 - val_loss: 0.6670 - val_accuracy: 0.7777
Epoch 98/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7524 - accuracy: 0.7434 - val_loss: 0.6632 - val_accuracy: 0.7767
Epoch 99/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7556 - accuracy: 0.7419 - val_loss: 0.6816 - val_accuracy: 0.7744
Epoch 100/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7633 - accuracy: 0.7395 - val_loss: 0.7147 - val_accurac

Epoch 150/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7433 - accuracy: 0.7445 - val_loss: 0.6844 - val_accuracy: 0.7674
Epoch 151/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7398 - accuracy: 0.7468 - val_loss: 0.6613 - val_accuracy: 0.7849
Epoch 152/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7517 - accuracy: 0.7477 - val_loss: 0.7072 - val_accuracy: 0.7641
Epoch 153/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7504 - accuracy: 0.7438 - val_loss: 0.6897 - val_accuracy: 0.7652
Epoch 154/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7574 - accuracy: 0.7416 - val_loss: 0.6816 - val_accuracy: 0.7764
Epoch 155/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7503 - accuracy: 0.7442 - val_loss: 0.7245 - val_accuracy: 0.7517
Epoch 156/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7463 - accuracy: 0.7435 - val_loss: 0

Epoch 206/400
782/782 [==============================] - 27s 35ms/step - loss: 0.7378 - accuracy: 0.7487 - val_loss: 0.6701 - val_accuracy: 0.7780
Epoch 207/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7586 - accuracy: 0.7401 - val_loss: 0.7092 - val_accuracy: 0.7687
Epoch 208/400
782/782 [==============================] - 27s 35ms/step - loss: 0.7490 - accuracy: 0.7458 - val_loss: 0.6423 - val_accuracy: 0.7862
Epoch 209/400
782/782 [==============================] - 27s 35ms/step - loss: 0.7503 - accuracy: 0.7425 - val_loss: 0.6921 - val_accuracy: 0.7672
Epoch 210/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7384 - accuracy: 0.7495 - val_loss: 0.6607 - val_accuracy: 0.7805
Epoch 211/400
782/782 [==============================] - 27s 35ms/step - loss: 0.7428 - accuracy: 0.7471 - val_loss: 0.6638 - val_accuracy: 0.7782
Epoch 212/400
782/782 [==============================] - 27s 35ms/step - loss: 0.7425 - accuracy: 0.7468 - val_loss: 0

Epoch 262/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7395 - accuracy: 0.7497 - val_loss: 0.7220 - val_accuracy: 0.7565
Epoch 263/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7356 - accuracy: 0.7521 - val_loss: 0.6603 - val_accuracy: 0.7833
Epoch 264/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7479 - accuracy: 0.7446 - val_loss: 0.7012 - val_accuracy: 0.7689
Epoch 265/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7457 - accuracy: 0.7468 - val_loss: 0.6551 - val_accuracy: 0.7828
Epoch 266/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7420 - accuracy: 0.7491 - val_loss: 0.7313 - val_accuracy: 0.7600
Epoch 267/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7445 - accuracy: 0.7485 - val_loss: 0.6477 - val_accuracy: 0.7815
Epoch 268/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7496 - accuracy: 0.7471 - val_loss: 0

Epoch 318/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7489 - accuracy: 0.7456 - val_loss: 0.7043 - val_accuracy: 0.7667
Epoch 319/400
782/782 [==============================] - 28s 35ms/step - loss: 0.7515 - accuracy: 0.7458 - val_loss: 0.6660 - val_accuracy: 0.7778
Epoch 320/400
782/782 [==============================] - 27s 35ms/step - loss: 0.7372 - accuracy: 0.7483 - val_loss: 0.6990 - val_accuracy: 0.7661
Epoch 321/400
782/782 [==============================] - 27s 35ms/step - loss: 0.7415 - accuracy: 0.7494 - val_loss: 0.6584 - val_accuracy: 0.7855
Epoch 322/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7495 - accuracy: 0.7455 - val_loss: 0.6878 - val_accuracy: 0.7728
Epoch 323/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7422 - accuracy: 0.7474 - val_loss: 0.6733 - val_accuracy: 0.7742
Epoch 324/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7448 - accuracy: 0.7457 - val_loss: 0

Epoch 374/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7480 - accuracy: 0.7481 - val_loss: 0.6821 - val_accuracy: 0.7683
Epoch 375/400
782/782 [==============================] - 27s 34ms/step - loss: 0.7525 - accuracy: 0.7449 - val_loss: 0.6754 - val_accuracy: 0.7784
Epoch 376/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7507 - accuracy: 0.7469 - val_loss: 0.6964 - val_accuracy: 0.7672
Epoch 377/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7482 - accuracy: 0.7462 - val_loss: 0.6762 - val_accuracy: 0.7762
Epoch 378/400
782/782 [==============================] - 26s 33ms/step - loss: 0.7460 - accuracy: 0.7473 - val_loss: 0.6545 - val_accuracy: 0.7816
Epoch 379/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7548 - accuracy: 0.7427 - val_loss: 0.6713 - val_accuracy: 0.7783
Epoch 380/400
782/782 [==============================] - 26s 34ms/step - loss: 0.7473 - accuracy: 0.7469 - val_loss: 0

In [14]:
# load the pre-saved model
model_10 = load_model('model_10_tiny_Base-CubeAI_Dual_3Conv.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_tiny_Base-CubeAI_Dual_3Conv_PTQ'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_Dual_3Conv_PTQ is {}%".format(accuracy_score*100))

313/313 [==============================] - 1s 4ms/step - loss: 0.6978 - accuracy: 0.7638
Test accuracy: 0.7638000249862671


INFO:tensorflow:Assets written to: /tmp/tmpojnlx8_a/assets


INFO:tensorflow:Assets written to: /tmp/tmpojnlx8_a/assets
/home/themandalorian/anaconda3/envs/mlonmcu_lab2/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-29 21:54:56.493528: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-29 21:54:56.493564: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-29 21:54:56.494771: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpojnlx8_a
2023-05-29 21:54:56.498400: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-29 21:54:56.498419: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpojnlx8_a
2023-05-29 21:54:56.506054: I tensorf

== Input details ==
name: serving_default_conv2d_52_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of model_10_tiny_Base-CubeAI_Dual_3Conv_PTQ is 76.62%


In [15]:
c_model_name = 'cifar10_tiny_base_cubeAI_dual_3conv_ptq'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model, c_model_name))

In [ ]:
model_10 = Sequential()
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# compile model
opt = SGD(lr=0.001, momentum=0.9)

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_callback = ModelCheckpoint(filepath='model_10_tiny_Base-CubeAI_Dual_3Conv_QAT.h5', save_freq='epoch')

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=400,
                      callbacks=[checkpoint_callback],validation_data=(x_test, y_test))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 32, 32, 3)        3         
 er)                                                             
                                                                 
 quant_conv2d_58 (QuantizeWr  (None, 32, 32, 16)       483       
 apperV2)                                                        
                                                                 
 quant_conv2d_59 (QuantizeWr  (None, 32, 32, 16)       2355      
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_34 (Qua  (None, 16, 16, 16)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_dropout_43 (QuantizeW  (None, 16, 16, 16)     

In [ ]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_tiny_Base-CubeAI_Dual_3Conv_QAT.tflite", "wb").write(tflite_model_quant_int8_qat)

In [ ]:
tflite_interpreter = tf.lite.Interpreter("model_10_tiny_Base-CubeAI_Dual_3Conv_QAT.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_tiny_Base-CubeAI_Dual_3Conv_QAT.tflite is {}%".format(accuracy_score*100))

c_model_name = 'cifar10_tiny_base_cubeAI_dual_3conv_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(tflite_model_quant_int8_qat, c_model_name))